In [1]:
# @formatter:off
import random

import numpy as np
import sklearn.ensemble
%load_ext autoreload
%autoreload 2
# @formatter:on
import os

os.chdir('../')
print(os.getcwd())

/home/viktor/workspace/smaite


In [2]:
#@formatter:off
files = !ls data/*csv.jsonl
#@formatter:on
files = [f for f in files if 'sample' not in f and 'augmented' not in f and 'processed' not in f]
assert len(files) == 8

In [11]:
#@formatter:off
snippet_files = !ls data/*-google-results-sample.jsonl | grep -v snippets
#@formatter:on

In [12]:
print(snippet_files)
files

['data/fullfact-google-results-sample.jsonl']


['data/bbc.csv.jsonl',
 'data/climatefeedback.csv.jsonl',
 'data/factcheck.csv.jsonl',
 'data/fullfact.csv.jsonl',
 'data/nytimes.csv.jsonl',
 'data/politifact.csv.jsonl',
 'data/snopes.csv.jsonl',
 'data/washingtonpost.csv.jsonl']

In [5]:
import json
from tqdm import tqdm

from unidecode import unidecode


def load_json(d):
    try:
        return json.loads(d, strict=False)
    except Exception as e:
        print("FEK")
        print(d)
        print(unidecode(d))
        return json.loads(unidecode(d), strict=False)


def load_jsonl(path: str):
    print("loading", path)
    with open(path) as f:
        return [load_json(d) for d in tqdm(f.readlines())]


snippet_datasets = [load_jsonl(p) for p in snippet_files]
datasets = [load_jsonl(p) for p in files]

loading data/fullfact-google-results-sample.jsonl


100%|██████████| 4061/4061 [00:02<00:00, 1625.75it/s]


loading data/snippets-fullfact-google-results-sample.jsonl


100%|██████████| 4061/4061 [00:00<00:00, 47509.26it/s]


loading data/bbc.csv.jsonl


...

100%|██████████| 5400/5400 [00:00<00:00, 35734.16it/s]


loading data/snopes.csv.jsonl


100%|██████████| 4499/4499 [00:00<00:00, 6259.51it/s]


loading data/washingtonpost.csv.jsonl


100%|██████████| 1416/1416 [00:00<00:00, 1552.96it/s]


In [13]:
snippet_map = {
    k: files.index(k.replace('-google-results-sample.jsonl', '.csv.jsonl')) for k in snippet_files
}
snippet_map

['data/bbc.csv.jsonl', 'data/climatefeedback.csv.jsonl', 'data/factcheck.csv.jsonl', 'data/fullfact.csv.jsonl', 'data/nytimes.csv.jsonl', 'data/politifact.csv.jsonl', 'data/snopes.csv.jsonl', 'data/washingtonpost.csv.jsonl']


{'data/fullfact-google-results-sample.jsonl': 3}

In [14]:
def process_snippet(d, claim_ds):
    if d['results'] and d['results'].get('items', None):
        snippets_formatted = '\n'.join(
            t.get('snippet', '').split('...', 1)[-1].replace('\xa0...', '').replace('...', '...\n...').strip() for t in d['results']['items']).strip()
    else:
        snippets_formatted = ''
    # print(snippets_formatted)
    claim = claim_ds[d['id']]['text']
    explanataion = claim_ds[d['id']]['cR_textualRating']
    text_article = claim_ds[d['id']]['text_article']
    return {
        'input_snippet': '\n'.join((claim, snippets_formatted)),
        'input_article': f"{claim}\n{text_article}",
        'target': explanataion
    }


ff_map = {d['id']: d for d in datasets[3]}
processed_fullfact_snippets = [process_snippet(s, ff_map) for s in snippet_datasets[0]]

In [15]:
from handystuff.loaders import write_jsonl


def write_out_ds(ds, f):
    f = f.replace('data/', 'data/snippets-')
    write_jsonl(ds, f)

write_out_ds(processed_fullfact_snippets, snippet_files[0])
